In [1]:
import pandas as pd
import numpy as np
import binet as bnt

pd.options.display.float_format = '{:,.2f}'.format

#### Import data

In [2]:
#Either directly from Atlas of Economic Complexity
wt, pr, co = bnt.trade_data('hs02')
wt = wt[(wt.ccode != 'xxa')&(wt.ccode != 'xxb')]

#Or from file if you have it.
# wt = pd.read_csv('wt.csv')
# pr = pd.read_csv('pr.csv')
# co = pd.read_csv('co.csv')

Retrieving trade data for hs02


In [3]:
#Select most important countries and products just to speed up computation
select_countries = list(wt.groupby('ccode').count().sort_values(by = 'pcode').tail(50).index)
select_products = list(wt.groupby('pcode').count().sort_values(by = 'ccode').tail(400).index)
wt_s = wt.loc[(wt.ccode.isin(select_countries))&(wt.pcode.isin(select_products))]

### binet.calculateRCA()

In [4]:
#Function bnt.calculateRCA takes data from only one year. If given data of many years it will just pool it together.
x = bnt.calculateRCA(wt_s.loc[wt_s.year == 2003], c = 'ccode', p = 'pcode', x = 'x')
x.sort_values(by = ['ccode','pcode']).head(2)

,ccode,pcode,x,RCA
1,are,106,"7,809,968.00",5.05
51,are,207,"2,616,744.08",0.08


### calculateRCA_by_year(), definition and examples:

In [5]:
#The function calculateRCA_by_year takes care of the year column. See examples below and compare with bnt.calculateRCA

def calculateRCA_by_year(data,y ='',c='',p='',x='',shares=False, log_terms = False):
    '''
    This function handles input data from more than one year.
    Returns the RCA expressed in data. All RCA values belong to a country-product-year.
    Parameters
    ----------
    data : pandas.DataFrame
        Raw data. It has year,source,target,volume (trade, number of people etc.).
    y,c,p,x : str (optional)
        Labels of the columns in data used for source,target,volume
    shares : boolean (False)
        If True it will also return the shares used to calculate the RCA
    log_terms: boolean(False)
        If True it instead returns the log exports log(x), log 'baseline term' log(\sum_c x_{cpy}  \sum_p x_{cpy} / \sum_c \sum_p x_{cpy})
    and log(RCA), which is by definition the diference of these two.
    Returns
    -------
    RCA : pandas.DataFrame
        Table with the RCAs, with the columns c,p,x,RCA
        If shares is True it includes:
            s_c : Share of X_cp over X_c
            s_p : Share of X_cp over X_p
        If log_terms is True, it instead includes:
            log(x) : log of exports
            T : log of the baseline term, which is market size of product * market size of country / total world trade
            log(RCA) : log of RCA computed as log(x) - T
            
    '''
    y = data.columns.values[0] if y == '' else y
    c = data.columns.values[1] if c == '' else c
    p = data.columns.values[2] if p == '' else p
    x = data.columns.values[3] if x == '' else x
    data_ = data[[y,c,p,x]]
    
    data_ = pd.merge(data_,data_.groupby([c,y]).sum()[[x]].rename(columns={x:x+'_'+c+'_'+y}).reset_index()
               ,how='inner',left_on=[y,c],right_on=[y,c]) #This is Tc
    data_ = pd.merge(data_,data_.groupby([p,y]).sum()[[x]].rename(columns={x:x+'_'+p+'_'+y}).reset_index()
                  ,how='inner',left_on=[y,p],right_on=[y,p])
    data_ = pd.merge(data_,data_.groupby(y).sum()[[x]].rename(columns={x:x+'_'+y}).reset_index()
                  ,how='inner',left_on=y,right_on=y)

    data_['RCA'] = (data_[x].astype(float)/data_[x+'_'+p+'_'+y].astype(float))/(data_[x+'_'+c+'_'+y].astype(float)/data_[x+'_'+y].astype(float))

    if shares:
        data_['s_'+c] = (data_[x].astype(float)/data_[x+'_'+c+'_'+y].astype(float)) 
        data_['s_'+p] = (data_[x].astype(float)/data_[x+'_'+p+'_'+y].astype(float))
        return data_[[y,c,p,x,'RCA','s_'+c,'s_'+p]]
    if log_terms:
        data_['log(x)'] = np.log10(data_[x].astype(float))
        data_['T'] = -np.log10((1/data_[x+'_'+p+'_'+y].astype(float))/(data_[x+'_'+c+'_'+y].astype(float)/data_[x+'_'+y].astype(float)))
        data_['log(RCA)'] = data_['log(x)'] - data_['T']
        return data_[[y,c,p,x,'RCA','log(x)','T','log(RCA)']]
    return data_[[y,c,p,x,'RCA']]


x2 = calculateRCA_by_year(wt_s,y ='year',c='ccode',p='pcode',x='x')
x2.sort_values(by = ['ccode','pcode']).head(20)

,year,ccode,pcode,x,RCA
1,2003,are,106,"7,809,968.00",5.05
19986,2004,are,106,"12,618,133.00",6.60
39976,2005,are,106,"12,317,379.02",3.86
59967,2006,are,106,"7,622,588.00",3.05
79962,2007,are,106,"10,592,555.23",2.33
99957,2008,are,106,"14,286,585.67",2.54
119950,2009,are,106,"10,916,775.00",3.45
139942,2010,are,106,"28,961,855.00",5.12
159934,2011,are,106,"27,231,668.00",4.28
179925,2012,are,106,"21,496,513.00",3.38


In [6]:
#Additionaly, the function calculateRCA_by_year has the option to compute log terms, which are really useful.
#These log terms are log(x), log(RCA) and T = log(x) - log(RCA)
x2 = calculateRCA_by_year(wt_s,y ='year',c='ccode',p='pcode',x='x', log_terms=True)
x2.sort_values(by = ['ccode','pcode']).head(20)

,year,ccode,pcode,x,RCA,log(x),T,log(RCA)
1,2003,are,106,"7,809,968.00",5.05,6.89,6.19,0.70
19986,2004,are,106,"12,618,133.00",6.60,7.10,6.28,0.82
39976,2005,are,106,"12,317,379.02",3.86,7.09,6.50,0.59
59967,2006,are,106,"7,622,588.00",3.05,6.88,6.40,0.48
79962,2007,are,106,"10,592,555.23",2.33,7.03,6.66,0.37
99957,2008,are,106,"14,286,585.67",2.54,7.15,6.75,0.40
119950,2009,are,106,"10,916,775.00",3.45,7.04,6.50,0.54
139942,2010,are,106,"28,961,855.00",5.12,7.46,6.75,0.71
159934,2011,are,106,"27,231,668.00",4.28,7.44,6.80,0.63
179925,2012,are,106,"21,496,513.00",3.38,7.33,6.80,0.53


### pRCA function. definition and example

In [7]:
#The function calculatepRCA computes pRCA by fitting k-nearest neighbors.
#It tells you the chances to have RCA > 1 next year, given log(x) and T.

from sklearn import neighbors

def calculatepRCA(data, y ='',c='',p='',x=''):
    '''
    Returns the pRCA from data. pRCA is the probability that (RCA_{y+1} > 1) given the volume of exports (x_{cpy}),
    and the 'baseline term' (\sum_c x_{cpy}  \sum_p x_{cpy} / \sum_c \sum_p x_{cpy}).
    It is computed using k-nearest neighbors, in the space of log exports and log baseline term.
    Parameters
    ----------
    data : pandas.DataFrame
        Raw data. It has source,target,volume (trade, number of people etc.).
    y,c,p,x : str (optional)
        Labels of the columns in data used for source,target,volume
    Returns
    -------
    RCA : pandas.DataFrame
        Table with the RCAs, with the columns c,p,x,RCA
        If shares is True it also includes:
            s_c : Share of X_cp over X_c
            s_p : Share of X_cp over X_p
    '''
    df = calculateRCA_by_year(data,y ='year',c='ccode',p='pcode',x='x',log_terms = True)
        
    #Compute (RCA > 1) next year and merge it
    df_ = df.copy()
    df_['year'] = df_['year'] - 1
    df_['RCA_y+1'] = (df_['log(RCA)'] > 0).astype(int)
    df_ = df_[['year','ccode','pcode','RCA_y+1']]
    df = df.merge(df_)
    
    #Prepare dataset for knn and fit
    M = df[['log(x)','T','RCA_y+1']].as_matrix()
    X, y = M[:,:2], M[:, 2] 
    knn = neighbors.KNeighborsRegressor(n_neighbors = 200, weights = 'uniform').fit(X, y)

    #To avoid memory error, compute predictions in split X. Predictions are output pRCA
    pRCA = np.array([])
    for x in np.array_split(X, 10):
        pRCA = np.append(pRCA, knn.predict(x))
    df['pRCA'] = pRCA
    
    return df

x3 = calculatepRCA(wt_s, y ='year',c='ccode',p='pcode',x='x')
x3.sample(10)

,year,ccode,pcode,x,RCA,log(x),T,log(RCA),RCA_y+1,pRCA
56246,2005,usa,8521,"332,841,966.58",0.19,8.52,9.24,-0.72,0,0.00
32868,2004,ind,8302,"154,987,701.50",1.08,8.19,8.16,0.04,1,0.64
120094,2009,irl,302,"121,482,247.88",1.09,8.08,8.05,0.04,1,0.73
73910,2006,blx,8429,"2,377,562,048.65",2.13,9.38,9.05,0.33,1,0.98
156213,2010,dnk,8523,"21,411,384.40",0.14,7.33,8.20,-0.87,0,0.00
918,2003,fin,712,"55,534.52",0.01,4.74,6.97,-2.23,0,0.00
27694,2004,hun,5806,"5,058,421.10",0.30,6.70,7.23,-0.53,0,0.01
197100,2012,rus,8543,"121,770,450.79",0.22,8.09,8.75,-0.66,0,0.00
149834,2010,swe,6405,"15,263,332.20",0.47,7.18,7.51,-0.33,0,0.02
68511,2006,aut,6117,"14,643,380.58",0.60,7.17,7.39,-0.22,0,0.06
